# Build a Retrieval Augmented Generation (RAG) App
One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.

This tutorial will show how to build a simple Q&A application over a text data source. Along the way we’ll go over a typical Q&A architecture and highlight additional resources for more advanced Q&A techniques. We’ll also see how LangSmith can help us trace and understand our application. LangSmith will become increasingly helpful as our application grows in complexity.

If you're already familiar with basic retrieval, you might also be interested in this [high-level overview of different retrieval techinques0(https://python.langchain.com/v0.2/docs/concepts/#retrieval).

## What is RAG?
RAG is a technique for augmenting LLM knowledge with additional data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

LangChain has a number of components designed to help build Q&A applications, and RAG applications more generally.

Note: Here we focus on Q&A for unstructured data. If you are interested for RAG over structured data, check out our tutorial on doing [question/answering over SQL data](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/).

## Concepts
A typical RAG application has two main components:

__Indexing__: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

__Retrieval and generation__: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

The most common full sequence from raw data to answer looks like:

### Indexing
1. Load: First we need to load our data. This is done with [Document Loaders](https://python.langchain.com/v0.2/docs/concepts/#document-loaders).
2. Split: [Text splitters](https://python.langchain.com/v0.2/docs/concepts/#text-splitters) break large `Documents` into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won't fit in a model's finite context window.
3. Store: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a [VectorStore](https://python.langchain.com/v0.2/docs/concepts/#vectorstores) and [Embeddings](https://python.langchain.com/v0.2/docs/concepts/#embedding-models) model.

### Retrieval and generation
4. Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/v0.2/docs/concepts/#retrievers).
5. Generate: A [ChatModel](https://python.langchain.com/v0.2/docs/concepts/#chat-models) / [LLM](https://python.langchain.com/v0.2/docs/concepts/#llms) produces an answer using a prompt that includes the question and the retrieved data

## Setup

### Installation
This tutorial requires these langchain dependencies:
```bash
pip install langchain langchain_community langchain_chroma
```

A non-comprehensive list of dependencies used in this examples are listed here
```bash

```

### LangSmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

After you sign up at the link above, make sure to set your environment variables to start logging traces:
```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

## Preview
In this guide we’ll build a QA app over as website. The specific website we will use is the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng, which allows us to ask questions about the contents of the post.

We can create a simple indexing pipeline and RAG chain to do this in ~20 lines of code:

### Setting credentials with python-dot-env
Load credentials from a .env file and the [python-dotenv package](https://pypi.org/project/python-dotenv/)

In [3]:
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["OPENAI_API_KEY"]

We will use the OpenAI `gpt-3.5-turbo-0125` model in this example

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [13]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print(len(docs))
print(docs[0].metadata)

1
{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [22]:
splits[0]

Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\

In [23]:
splits[1]

Document(page_content='Memory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})

In [24]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [27]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique that involves breaking down complex tasks into smaller and simpler steps to facilitate problem-solving. It can be achieved through prompting methods like Chain of Thought or Tree of Thoughts, which guide models to think step by step. This approach helps in making big tasks more manageable and enhances model performance on complex tasks.'

API Reference: [WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html) | [StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) | [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) | [OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html) | [RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [29]:
vectorstore.delete_collection()

## Detailed walkthrough
Let’s go through the above code step-by-step to really understand what’s going on.

### 1. Indexing: Load
We need to first load the blog post contents. We can use [DocumentLoaders](https://python.langchain.com/v0.2/docs/concepts/#document-loaders) for this, which are objects that load in data from a source and return a list of Documents. A [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) is an object with some page_content (str) and metadata (dict).

In this case we’ll use the [WebBaseLoader](https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/), which uses `urllib` to load HTML from web URLs and `BeautifulSoup` to parse it to text. We can customize the HTML -> text parsing by passing in parameters to the `BeautifulSoup` parser via `bs_kwargs` (see [BeautifulSoup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we’ll remove all others.



In [33]:
import bs4
from langchain_community.document_loaders import WebBaseLoader


bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content")) # Only keep post title, headers, and content from the full HTML.

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content) # Number of characters in content

43131

In [35]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### Go deeper
`DocumentLoader`: Object that loads data from a source as list of Documents.

* [Docs](https://python.langchain.com/v0.2/docs/how_to/#document-loaders): Detailed documentation on how to use `DocumentLoaders`.
* [Integrations](https://python.langchain.com/v0.2/docs/integrations/document_loaders/): 160+ integrations to choose from.
* [Interface](https://api.python.langchain.com/en/latest/document_loaders/langchain_core.document_loaders.base.BaseLoader.html): API reference  for the base interface.

## 2. Indexing: Split
Our loaded document is over 42k characters long. This is too long to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the `Document` into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the blog post at run time.

In this case we’ll split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/), which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # 1k characters in each chunk
    chunk_overlap=200,     # 200 characters overlap across chunks to prevent separating statements from important context
    add_start_index=True   # metadata preserves the starting character number from original document
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

In [38]:
len(all_splits[0].page_content)

969

In [39]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

### Go deeper
`TextSplitter`: Object that splits a list of `Documents` into smaller chunks. Subclass of `DocumentTransformers`.

* Learn more about splitting text using different methods by reading the [how-to docs](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)
* [Code (py or js)](https://python.langchain.com/v0.2/docs/integrations/document_loaders/source_code/)
* [Scientific papers](https://python.langchain.com/v0.2/docs/integrations/document_loaders/grobid/)
* [Interface](https://api.python.langchain.com/en/latest/base/langchain_text_splitters.base.TextSplitter.html): API reference for the base interface.

`DocumentTransformer`: Object that performs a transformation on a list of `Document` objects.

* [Docs](https://python.langchain.com/v0.2/docs/how_to/#text-splitters): Detailed documentation on how to use DocumentTransformers
* [Integrations](https://python.langchain.com/v0.2/docs/integrations/document_transformers/)
* [Interface](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html): API reference for the base interface.

## 3. Indexing: Store
Now we need to index our 66 text chunks so that we can search over them at runtime. The most common way to do this is to __embed the contents of each document split and insert these embeddings into a vector database (or vector store).__ 

When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding. The simplest similarity measure is __cosine similarity__ — we measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors).

We can embed and store all of our document splits in a single command using the [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) vector store and [OpenAIEmbeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/) model.

In [41]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())

API Reference: [OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)

[Embeddings](https://python.langchain.com/v0.2/docs/how_to/embed_text/): Wrapper around a text embedding model, used for converting text to embeddings.

[VectorStore](https://python.langchain.com/v0.2/docs/how_to/vectorstores/): Wrapper around a vector database, used for storing and querying embeddings.

This completes the __Indexing__ portion of the pipeline. At this point we have a query-able vector store containing the chunked contents of our blog post. Given a user question, we should ideally be able to return the snippets of the blog post that answer the question.

## 4. Retrieval and Generation: Retrieve
Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

First we need to define our logic for searching over documents. LangChain defines a [Retriever](https://python.langchain.com/v0.2/docs/concepts/#retrievers/) interface which wraps an index that can return relevant `Documents` given a string query.

The most common type of `Retriever` is the [VectorStoreRetriever](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/), which uses the similarity search capabilities of a vector store to facilitate retrieval. Any `VectorStore` can easily be turned into a `Retriever` with `VectorStore.as_retriever()`:

In [44]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6}) 

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?") # Retrieve a chunk that is most similar to the query by cosine similarity score

len(retrieved_docs)

6

In [45]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


### Go deeper
Vector stores are commonly used for retrieval, but there are other ways to do retrieval, too.

`Retriever`: An object that returns `Documents` given a text query

* [Docs](https://python.langchain.com/v0.2/docs/how_to/#retrievers): Further documentation on the interface and built-in retrieval techniques. Some of which include:
    * `MultiQueryRetriever` generates [variants of the input question](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/) to improve retrieval hit rate.
    * `MultiVectorRetriever` instead generates [variants of the embeddings](https://python.langchain.com/v0.2/docs/how_to/multi_vector/), also in order to improve retrieval hit rate.
    * `Max marginal relevance` selects for [relevance and diversity](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) among the retrieved documents to avoid passing in duplicate context.
    * Documents can be filtered during vector store retrieval using metadata filters, such as with a [Self Query Retriever](https://python.langchain.com/v0.2/docs/how_to/self_query/).
* [Integrations](https://python.langchain.com/v0.2/docs/integrations/retrievers/): Integrations with retrieval services.
* [Interface](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html): API reference for the base interface.

## 5. Retrieval and Generation: Generate
Let’s put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output.

We’ll use the gpt-3.5-turbo OpenAI chat model, but any LangChain `LLM` or `ChatModel` could be substituted in.

We’ll use a prompt for RAG that is checked into the LangChain prompt [hub](https://smith.langchain.com/hub/rlm/rag-prompt).



In [47]:
from langchain import hub


# From https://smith.langchain.com/hub/rlm/rag-prompt
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
# If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Question: {question} 
# Context: {context} 
# Answer:
prompt = hub.pull("rlm/rag-prompt")

# Fill in the PromptTemplate
example_messages = prompt.invoke(
    {"context": "filler context",
     "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [48]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


We’ll use the [LCEL Runnable protocol](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) to define the chain, allowing us to
* pipe together components and functions in a transparent way
* automatically trace our chain in LangSmith
* get streaming, async, and batched calling out of the box.

Here is the implementation:

In [51]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# This contexts a chain of Runnables i.e. RunnableSequence
# Runnables all implement the .invoke, .stream and .batch methods
rag_chain = (
    # Pass the question to the retriever, which genereates Document objects
    # Then pass those to format_docs to generate strings
    # The context is a list of strings
    # RunnablePassthrough() passes through the input question unchanged
    {"context": retriever | format_docs, "question": RunnablePassthrough()}  # Cast to RunnablParallel
    | prompt             # Cast to Runnable
    | llm                # Runs the inferenace
    | StrOutputParser()  # Picks the string context out of the LLM's output message
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is a method of breaking down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into multiple manageable tasks and shedding light on the model's thinking process. This can be achieved through simple prompting, task-specific instructions, or human inputs.

Let's dissect the LCEL to understand what's going on.

First: each of these components (`retriever`, `prompt`, `llm`, etc.) are instances of [Runnable](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel). This means that they implement the same methods-- such as sync and async `.invoke`, `.stream`, or `.batch`-- which makes them easier to connect together. They can be connected into a [RunnableSequence](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html)-- another Runnable-- via the `|` operator.

LangChain will automatically cast certain objects to runnables when met with the `|` operator. Here, `format_docs` is cast to a `RunnableLambda`, and the dict with `"context"` and `"question"` is cast to a `RunnableParallel`. The details are less important than the bigger point, which is that each object is a Runnable.

Let's trace how the input question flows through the above runnables.

As we've seen above, the input to prompt is expected to be a `dict` with keys `"context"` and `"question"`. So the first element of this chain builds runnables that will calculate both of these from the input question:
* `retriever | format_docs` passes the question through the retriever, generating Document objects, and then to `format_docs` to generate strings;
*  `RunnablePassthrough()` passes through the input question unchanged.

That is, if you constructed

```python
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)
```
Then `chain.invoke(question)` would build a formatted prompt, ready for inference. (Note: when developing with LCEL, it can be practical to test with sub-chains like this.)

The last steps of the chain are `llm`, which runs the inference, and `StrOutputParser()`, which just plucks the string content out of the LLM's output message.

## Built-in chains
If preferred, LangChain includes convenience functions that implement the above LCEL. We compose two functions:

* [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) specifies how retrieved context is fed into a prompt and LLM. In this case, we will "stuff" the contents into the prompt -- i.e., we will include all retrieved context without any summarization or other processing. It largely implements our above `rag_chain`, with input keys `context` and `input`-- it generates an answer using retrieved context and query.
* [create_retrieval_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html) adds the retrieval step and propagates the retrieved context through the chain, providing it alongside the final answer. It has input key `input`, and includes `input`, `context`, and `answer` in its output.

In [52]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# specifies how retrieved context is fed into a prompt and LLM. In this case, we will "stuff" the contents into the prompt 
# -- i.e., we will include all retrieved context without any summarization or other processing.
question_answer_chain = create_stuff_documents_chain(llm, prompt) 

# adds the retrieval step and propagates the retrieved context through the chain, providing it alongside the final answer. 
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts help models decompose hard tasks into multiple manageable tasks. This process aids in enhancing model performance on complex tasks by transforming big tasks into smaller, more achievable steps.


Returning sources
Often in Q&A applications it's important to show users the sources that were used to generate the answer.

LangChain's built-in `create_retrieval_chain` will propagate retrieved source documents through to the output in the `"context"` key:

In [53]:
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be 

### Go deeper
Go deeper
`ChatModel`: An LLM-backed chat model. Takes in a sequence of messages and returns a message.
[Docs]()
[Integrations](): 25+ integrations to choose from.
[Interface](): API reference for the base interface.

`LLM`: A text-in-text-out LLM. Takes in a string and returns a string.
[Docs](https://python.langchain.com/v0.2/docs/how_to/#llms):
[Integrations](https://python.langchain.com/v0.2/docs/integrations/llms/): 75+ integrations to choose from.
[Interface](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.llms.BaseLLM.html): API reference for the base interface.

See a guide on RAG with locally-running models [here](https://python.langchain.com/v0.2/docs/tutorials/local_rag/).

Customizing the prompt
As shown above, we can load prompts (e.g., this RAG prompt) from the prompt hub. The prompt can also be easily customized:

In [54]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down complex tasks into smaller and simpler steps to enhance model performance. It can be achieved through techniques like Chain of Thought and Tree of Thoughts to transform big tasks into manageable ones. Task decomposition can be done through simple prompting, task-specific instructions, or human inputs. Thanks for asking!'

## Next steps
We've covered the steps to build a basic Q&A app over data:

* Loading data with a Document Loader
* Chunking the indexed data with a Text Splitter to make it more easily usable by a model
* Embedding the data and storing the data in a vectorstore
* Retrieving the previously stored chunks in response to incoming questions
* Generating an answer using the retrieved chunks as context

There’s plenty of features, integrations, and extensions to explore in each of the above sections. Along from the Go deeper sources mentioned above, good next steps include:

* Return sources: Learn how to return source documents
* Streaming: Learn how to stream outputs and intermediate steps
* Add chat history: Learn how to add chat history to your app
* Retrieval conceptual guide: A high-level overview of specific retrieval techniques